In [1]:
from google.colab import drive
drive.mount('/content/drive/')

#!cp "/content/drive/My Drive/Dissertation/preprocessing.py" .
#!cp -r "/content/drive/My Drive/Dissertation/gym_maze" .
#!cp -r "/content/drive/My Drive/Dissertation/envs" .

Mounted at /content/drive/


In [2]:
# for inference, not continued training
def save_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/{name}" 

    torch.save({
      'meta_controller': model.meta_controller.state_dict(),
      'controller': model.controller.state_dict()
    }, path)

import copy
def load_model(model, name):
    path = f"/content/drive/My Drive/Dissertation/saved_models/{name}" 
    checkpoint = torch.load(path)

    model.meta_controller.load_state_dict(checkpoint['meta_controller'])
    model.meta_controller_target = copy.deepcopy(model.meta_controller)
    model.controller.load_state_dict(checkpoint['controller'])
    model.controller_target = copy.deepcopy(model.controller)

    model.eval()
    model.meta_controller.eval()
    model.controller.eval()

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import cv2
from PIL import Image

from IPython import display
plt.ion()

# if gpu is to be used
device = torch.device("cuda")

In [4]:
env = gym.make("Acrobot-v1")

---
### Helper functions

In [5]:
loss_plot = []
loss_plot_meta = []
def plot_durations(episode_durations):
    global loss_plot
    fig, axs = plt.subplots(3, figsize=(10,15))
    
    durations_t, durations = list(map(list, zip(*episode_durations)))
    durations = torch.tensor(durations, dtype=torch.float)
    
    fig.suptitle('Training')
    axs[0].set_xlabel('Episode')
    axs[0].set_ylabel('Reward')
    axs[1].set_xlabel('Steps')
    axs[1].set_ylabel('Loss')
    axs[2].set_xlabel('Steps')
    axs[2].set_ylabel('Loss (meta)')
    
    axs[0].plot(durations_t, durations.numpy())

    if len(loss_plot) > 0:
        durations_t, durations = list(map(list, zip(*loss_plot)))
        durations = torch.tensor(durations, dtype=torch.float)

        axs[1].plot(durations_t, durations.numpy())
    if len(loss_plot_meta) > 0:
        durations_t, durations = list(map(list, zip(*loss_plot_meta)))
        durations = torch.tensor(durations, dtype=torch.float)

        axs[2].plot(durations_t, durations.numpy())
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

---
### Code

In [6]:
# (state, action) -> (next_state, reward, done)
transition = namedtuple('transition', ('state', 'action', 'next_state', 'reward', 'done'))

# replay memory D with capacity N
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    # implemented as a cyclical queue
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

def soft_update(target, source, tau):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - tau) + param.data * tau
        )

def hard_update(target, source):
    for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)

In [7]:
BATCH_SIZE = 64
GAMMA = 0.99

class DQN(nn.Module):
    def __init__(self, inputs, outputs, mem_len = 100000):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(inputs, 128)
        self.fc2 = nn.Linear(128, 128)
        self.head = nn.Linear(128, outputs)
        
        self.memory = ReplayMemory(mem_len)

        self.n_actions = outputs
        self.steps_done = 0
        
        self.EPS_START = 1.0
        self.EPS_END = 0.01
        self.EPS_DECAY = 10000 # in number of steps
        self.TAU = 0.001

        self.policy_update = 2
        self.tot_updates = 0

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.head(x)
    
    def act(self, state, is_training):
        if is_training:
            eps_threshold = self.EPS_END + (self.EPS_START - self.EPS_END) * (1. - min(1., self.steps_done / self.EPS_DECAY))
            self.steps_done += 1

            # With probability eps select a random action
            if random.random() < eps_threshold:
                return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

        # otherwise select action = maxa Q∗(φ(st), a; θ)
        with torch.no_grad():
            return self(state).max(1)[1].view(1, 1)
    
    def experience_replay(self, optimizer, target):
        if len(self.memory) < BATCH_SIZE:
            return

        self.tot_updates += 1
        
        # in the form (state, action) -> (next_state, reward, done)
        transitions = self.memory.sample(BATCH_SIZE)
        batch = transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        next_state_batch = torch.cat(batch.next_state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        done_mask = np.array(batch.done)
        not_done_mask = torch.from_numpy(1 - done_mask).float().to(device)
        
        current_Q_values = self(state_batch).gather(1, action_batch)
        # Compute next Q value based on which goal gives max Q values
        # Detach variable from the current graph since we don't want gradients for next Q to propagated
        next_max_q = target(next_state_batch).detach().max(1)[0]
        next_Q_values = not_done_mask * next_max_q
        # Compute the target of the current Q values
        target_Q_values = reward_batch + (GAMMA * next_Q_values)
        # Compute Bellman error (using Huber loss)
        loss = F.smooth_l1_loss(current_Q_values, target_Q_values.unsqueeze(1))
        loss_val = loss.item()

        # Optimize the model
        optimizer.zero_grad()
        loss.backward()
        for param in self.parameters():
            param.grad.data.clamp_(-1, 1)
        optimizer.step()

        if self.tot_updates % self.policy_update == 0:
            soft_update(target, self, self.TAU)

        return loss_val
        
class HDQN(nn.Module):
    def __init__(self, inputs, outputs):
        super(HDQN, self).__init__()
        # Optimizer
        learning_rate = 2.5e-4
        
        # goal is left/right
        self.meta_controller = DQN(inputs, outputs).to(device)
        self.meta_controller_optimizer = optim.RMSprop(self.meta_controller.parameters(), lr=learning_rate)
        self.meta_controller_target = DQN(inputs, outputs, mem_len = 0).to(device)
        self.meta_controller_target.eval()
        
        # takes goal+state jointly
        self.controller = DQN(inputs + 1, outputs).to(device)
        self.controller_optimizer = optim.RMSprop(self.controller.parameters(), lr=learning_rate)
        self.controller_target = DQN(inputs + 1, outputs, mem_len = 0).to(device)
        self.controller_target.eval()

        self.controller.EPS_END = 0.0
        self.controller.EPS_DECAY = 1000
    
    def store_controller(self, *args):
        self.controller.memory.store(*args)
    
    def store_meta_controller(self, *args):
        self.meta_controller.memory.store(*args)
    
    def select_goal(self, external_observation, is_training):
        return self.meta_controller.act(external_observation, is_training)
        
    def select_action(self, joint_goal_obs, is_training):
        return self.controller.act(joint_goal_obs, is_training)
    
    def teach_meta_controller(self):
        return self.meta_controller.experience_replay(self.meta_controller_optimizer, self.meta_controller_target)

    def teach_controller(self):
        return self.controller.experience_replay(self.controller_optimizer, self.controller_target)

In [ ]:
def plot_norms(episode_durations):
    plt.figure(2, figsize=(10,10))
    
    x, ys = np.array(list(episode_durations.keys())), np.array(list(episode_durations.values()))
    
    plt.title('Action Prediction $\mu$ and $\pm \sigma$ interval')
    plt.xlabel('L2 Norm')
    plt.ylabel('Average Reward')
    
    mu = np.mean(ys, axis=1)
    plt.plot(x, mu)
    stds = np.std(ys, axis = 1)
    plt.fill_between(x, mu + stds , mu - stds, alpha=0.2)
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [8]:
SAVE_OFFSET = 10

def train_model():
    global SAVE_OFFSET
    # Get number of actions and observations from gym action space
    n_actions = env.action_space.n
    n_observations = env.observation_space.shape[0]

    # Initialize action-value function Q with random weights
    hdqnAgent = HDQN(n_observations, n_actions).to(device)

    max_episode_length = 500

    num_episodes = 5000 # M
    episode_durations = []

    steps = 0
    for i_episode in range(num_episodes):
        observation = env.reset()

        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        overall_reward = 0
        done = False
        episode_steps = 0
        while not done:
            # select a goal
            goal = hdqnAgent.select_goal(state, True)
            goal_i = goal.item()

            goal_done = False
            total_extrinsic = 0
            s_0 = state
            steps_until_goal = 0
            while not done and not goal_done:
                joint_goal_state = torch.cat([goal, state], axis=1)

                # Execute action a_t in emulator and observe reward r_t and image x_{t+1}
                action = hdqnAgent.select_action(joint_goal_state, True)
                action_i = action.item()

                observation, reward, done, _ = env.step(action_i)
                steps += 1

                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                episode_steps += 1
                
                extrinsic_reward = torch.tensor([reward], device=device)

                overall_reward += reward
                total_extrinsic += reward

                # preprocess φ_{t+1} = φ(s_{t+1})
                next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                joint_next_state = torch.cat([goal, next_state], axis=1)

                goal_done = (goal_i == action_i)
                if not goal_done:
                    steps_until_goal += 1
                else:
                    pass
                    #loss_plot.append((steps, steps_until_goal))

                intrinsic_reward = torch.tensor([1.0 if goal_done else 0], device=device)

                # Store transition (φt, at, rt, φt+1) in D
                hdqnAgent.store_controller(joint_goal_state, action, joint_next_state, intrinsic_reward, done)

                state = next_state

                loss = hdqnAgent.teach_controller()
                if loss is not None and i_episode % 50 == 0:
                    loss_plot.append((steps, loss))

            # Store transition for meta controller
            hdqnAgent.store_meta_controller(s_0, goal, next_state, torch.tensor([total_extrinsic], device=device), done)
            loss = hdqnAgent.teach_meta_controller()
            if loss is not None and i_episode % 50 == 0:
                loss_plot_meta.append((steps, loss))
        
        episode_durations.append((i_episode, overall_reward))
        #plot_durations(episode_durations)
        _, dur = list(map(list, zip(*episode_durations)))
        if len(dur) > 100:
            if i_episode % 100 == 0 and i_episode >= 1000 and np.mean(dur[-100:]) < -200.0:
                print(f"Failed to get lucky after 1000 eps, terminating... Avg: {np.mean(dur[-100:])}")
                return None # unlucky
            if i_episode % 300 == 0:
                print(f"Episode {i_episode}: {np.mean(dur[-100:])}")
            if np.mean(dur[-100:]) >= -90:
                print(f"Solved after {i_episode} episodes!")
                save_model(hdqnAgent, f"hdqn_acrobot_{SAVE_OFFSET}")
                SAVE_OFFSET += 1
                return hdqnAgent

    return None # did not train

In [ ]:
loss_plot = []
loss_plot_meta = []
#agent = train_model()

In [9]:
i = 10
while i < 11:
    loss_plot = []
    loss_plot_meta = []
    agent = train_model()
    if agent is not None:
        print(f"Num. {i} done!")
        i += 1

Episode 300: -145.29
Episode 600: -144.33
Episode 900: -126.0
Episode 1200: -97.58
Solved after 1361 episodes!
Num. 10 done!


In [13]:
state_max = torch.from_numpy(env.observation_space.high).to(device)
state_min = torch.from_numpy(env.observation_space.low).to(device)
def fgsm_attack(data, eps, data_grad):
    sign_data_grad = data_grad.sign()

    perturbed_data = data + eps * sign_data_grad * state_max

    clipped_perturbed_data = torch.max(torch.min(perturbed_data, state_max), state_min)

    return clipped_perturbed_data

def fgsm_goal(g_state, agent, eps, target, targetted):
    #g_state = torch.tensor(g_state, requires_grad=True)

    g_state_var = g_state.clone().detach().requires_grad_(True)

    # initial forward pass
    goal = agent.meta_controller(g_state_var)
    #goal = temp.max(1)[1].view(1, 1)

    if targetted:
        loss = F.smooth_l1_loss(goal, target)
    else:
        pass
        #loss = F.smooth_l1_loss(goal, temp.min(1)[1].view(1, 1).float())

    agent.meta_controller.zero_grad()

    # calc loss
    loss.backward()
    data_grad = g_state_var.grad.data

    # perturb state
    g_state_p = fgsm_attack(g_state, eps, data_grad)
    return agent.select_goal(g_state_p, False)

def fgsm_action(state, goal, agent, eps, target, targetted):
    #state = torch.tensor(state, requires_grad=True)
    state_var = state.clone().detach().requires_grad_(True)

    joint_goal_state = torch.cat([goal, state_var], 1).float()
    
    # initial forward pass
    action = agent.controller(joint_goal_state)
    #action = temp.max(1)[1].view(1, 1).float()

    if targetted:
        loss = F.smooth_l1_loss(action, target)
    else:
        pass
        #loss = F.smooth_l1_loss(action, temp.min(1)[1].view(1, 1).float())

    agent.controller.zero_grad()

    # calc loss
    loss.backward()
    data_grad = state_var.grad.data
    # perturb state
    state_p = fgsm_attack(state, eps, data_grad)

    joint_goal_state = torch.cat([goal, state_p], 1).float()
    return agent.select_action(joint_goal_state, False)

def apply_fgsm(agent, episode_durations, goal_attack, action_attack, targetted):
    TARGET_GOAL = torch.tensor([[0.0, 0.0, 0.0]], device=device, dtype=torch.float)
    TARGET_ACTION = torch.tensor([[0.0, 0.0, 0.0]], device=device, dtype=torch.float)

    agent.eval()
    agent.meta_controller.eval()
    agent.controller.eval()

    max_episode_length = 500

    num_episodes = 100

    for eps in np.arange(0.0, 0.031, 0.0025):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                
            episode_steps = 0
            done = False
            while not done:
                # select a goal
                if goal_attack:
                    goal = fgsm_goal(state, agent, eps, TARGET_GOAL, targetted)
                else:
                    goal = agent.select_goal(state, False)
                goal_i = goal.item()

                goal_done = False
                while not done and not goal_done:
                    joint_goal_state = torch.cat([goal, state], axis=1)

                    if action_attack:
                        action = fgsm_action(state, goal, agent, eps, TARGET_ACTION, targetted)
                    else:
                        action = agent.select_action(joint_goal_state, False)

                    action_i = action.item()
                    observation, reward, done, _ = env.step(action_i)

                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    goal_done = (goal_i == action_i)

                    state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[eps].append(overall_reward / num_episodes)

In [10]:
state_max = torch.from_numpy(env.observation_space.high).to(device)
def eval_model(hdqnAgent, episode_durations, goal_noise, action_noise, same_noise):
    hdqnAgent.eval()
    hdqnAgent.meta_controller.eval()
    hdqnAgent.controller.eval()

    max_episode_length = 500
    num_episodes = 100

    for l2norm in np.arange(0,0.31,0.03):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()

            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
            g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            noise = torch.FloatTensor(state.shape).uniform_(-l2norm/2, l2norm/2).to(device)
            if goal_noise:
                g_state = state + state_max * noise
                g_state = g_state.float()
            if action_noise:
                if same_noise:
                    state = state + state_max * noise
                else:
                    state = state + state_max * torch.FloatTensor(state.shape).uniform_(-l2norm/2, l2norm/2).to(device)
                state = state.float()

            episode_steps = 0
            done = False
            while not done:
                # select a goal
                goal = hdqnAgent.select_goal(g_state, False)
                goal_i = goal.item()

                goal_done = False
                while not done and not goal_done:
                    joint_goal_state = torch.cat([goal, state], axis=1)

                    action = hdqnAgent.select_action(joint_goal_state, False)
                    action_i = action.item()
                    observation, reward, done, _ = env.step(action_i)

                    overall_reward += reward

                    if max_episode_length and episode_steps >= max_episode_length - 1:
                        done = True
                    episode_steps += 1

                    goal_done = (goal_i == action_i)

                    state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
                    g_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

                    noise = torch.FloatTensor(state.shape).uniform_(-l2norm/2, l2norm/2).to(device)
                    if goal_noise:
                        g_state = state + state_max * noise
                        g_state = g_state.float()
                    if action_noise:
                        if same_noise:
                            state = state + state_max * noise
                        else:
                            state = state + state_max * torch.FloatTensor(state.shape).uniform_(-l2norm/2, l2norm/2).to(device)
                        state = state.float()

        episode_durations[l2norm].append(overall_reward / num_episodes)

In [ ]:
def plot_fgsm(episode_durations):
    plt.figure(2, figsize=(10,10))
    
    for kk in ['both', 'goal_only', 'action_only']:
        x, ys = np.array(list(episode_durations[kk].keys())), np.array(list(episode_durations[kk].values()))
        #plt.title('Action Prediction $\mu$ and $\pm \sigma$ interval')
        plt.xlabel('$\epsilon$')
        plt.ylabel('Average Reward')
        
        mu = np.mean(ys, axis=1)
        plt.plot(x, mu, label=kk)
        stds = np.std(ys, axis = 1)
        plt.fill_between(x, mu + stds , mu - stds, alpha=0.2)
    
    plt.legend()
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [14]:
targeted = {'both': {}, 'goal_only': {}, 'action_only': {}}
untargeted = {'both': {}, 'goal_only': {}, 'action_only': {}}
for eps in np.arange(0.0, 0.031, 0.0025):
    for x in ['both', 'goal_only', 'action_only']:
        targeted[x][eps] = []
        untargeted[x][eps] = []

n_actions = env.action_space.n
n_observations = env.observation_space.shape[0]

i = 0
while i < 11:
    #agent = train_model()
    agent = HDQN(n_observations, n_actions).to(device)
    load_model(agent, f"hdqn_acrobot_{i}")
    if agent is not None:
        apply_fgsm(agent, targeted['both'], True, True, True)
        apply_fgsm(agent, targeted['goal_only'], True, False, True)
        apply_fgsm(agent, targeted['action_only'], False, True, True)
        #apply_fgsm(agent, untargeted['both'], True, True, False)
        #apply_fgsm(agent, untargeted['goal_only'], True, False, False)
        #apply_fgsm(agent, untargeted['action_only'], False, True, False)
        print(i)
        print(f"Targeted: {targeted}")
        print(f"Untargeted: {untargeted}")
        #plot_fgsm(episode_durations)
        i += 1

#plot_fgsm(episode_durations)
print(f"Targeted: {targeted}")
print(f"Untargeted: {untargeted}")

0
Targeted: {'both': {0.0: [-95.42], 0.0025: [-97.86], 0.005: [-99.11], 0.0075: [-100.9], 0.01: [-96.07], 0.0125: [-114.32], 0.015: [-119.39], 0.0175: [-114.82], 0.02: [-117.03], 0.0225: [-127.63], 0.025: [-122.41], 0.0275: [-124.3], 0.03: [-146.24]}, 'goal_only': {0.0: [-92.32], 0.0025: [-96.31], 0.005: [-94.1], 0.0075: [-101.48], 0.01: [-91.48], 0.0125: [-96.54], 0.015: [-102.71], 0.0175: [-104.31], 0.02: [-115.98], 0.0225: [-105.39], 0.025: [-102.84], 0.0275: [-123.53], 0.03: [-123.79]}, 'action_only': {0.0: [-99.28], 0.0025: [-100.16], 0.005: [-95.38], 0.0075: [-102.32], 0.01: [-100.72], 0.0125: [-102.34], 0.015: [-97.2], 0.0175: [-96.13], 0.02: [-96.72], 0.0225: [-94.4], 0.025: [-96.36], 0.0275: [-96.48], 0.03: [-95.89]}}
Untargeted: {'both': {0.0: [], 0.0025: [], 0.005: [], 0.0075: [], 0.01: [], 0.0125: [], 0.015: [], 0.0175: [], 0.02: [], 0.0225: [], 0.025: [], 0.0275: [], 0.03: []}, 'goal_only': {0.0: [], 0.0025: [], 0.005: [], 0.0075: [], 0.01: [], 0.0125: [], 0.015: [], 0.017

In [11]:
same_noise = {}
diff_noise = {}
goal_only = {}
action_only = {}
for l2norm in np.arange(0,0.31,0.03):
    for i in [same_noise, diff_noise, goal_only, action_only]:
        i[l2norm] = []

n_actions = env.action_space.n
n_observations = env.observation_space.shape[0]

i = 10
while i < 11:
    #agent = train_model()
    agent = HDQN(n_observations, n_actions).to(device)
    load_model(agent, f"hdqn_acrobot_{i}")
    if agent is not None:
        # goal_attack, action_attack, same_noise
        eval_model(agent, same_noise, True, True, True)
        eval_model(agent, diff_noise, True, True, False)
        eval_model(agent, goal_only, True, False, False)
        eval_model(agent, action_only, False, True, False)
        print(i)
        print(f"same noise: {same_noise}")
        print(f"diff noise: {diff_noise}")
        print(f"goal only: {goal_only}")
        print(f"action only: {action_only}")
        i += 1

print(f"same noise: {same_noise}")
print(f"diff noise: {diff_noise}")
print(f"goal only: {goal_only}")
print(f"action only: {action_only}")

10
same noise: {0.0: [-91.89], 0.03: [-94.96], 0.06: [-99.97], 0.09: [-101.06], 0.12: [-107.54], 0.15: [-114.63], 0.18: [-117.84], 0.21: [-125.39], 0.24: [-141.21], 0.27: [-148.73], 0.3: [-153.65]}
diff noise: {0.0: [-91.46], 0.03: [-93.28], 0.06: [-99.33], 0.09: [-103.65], 0.12: [-110.83], 0.15: [-109.97], 0.18: [-122.57], 0.21: [-122.41], 0.24: [-140.41], 0.27: [-145.1], 0.3: [-162.54]}
goal only: {0.0: [-94.85], 0.03: [-96.59], 0.06: [-99.86], 0.09: [-104.94], 0.12: [-105.7], 0.15: [-110.61], 0.18: [-112.61], 0.21: [-120.17], 0.24: [-127.97], 0.27: [-137.54], 0.3: [-149.36]}
action only: {0.0: [-92.3], 0.03: [-99.47], 0.06: [-94.77], 0.09: [-94.28], 0.12: [-91.94], 0.15: [-96.1], 0.18: [-101.82], 0.21: [-98.42], 0.24: [-98.57], 0.27: [-100.67], 0.3: [-97.36]}
same noise: {0.0: [-91.89], 0.03: [-94.96], 0.06: [-99.97], 0.09: [-101.06], 0.12: [-107.54], 0.15: [-114.63], 0.18: [-117.84], 0.21: [-125.39], 0.24: [-141.21], 0.27: [-148.73], 0.3: [-153.65]}
diff noise: {0.0: [-91.46], 0.03